In [1]:
import sys 
sys.path.insert(0, "/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Uncertainty_Modeling")


import os
import json
import torch
import torch.optim as optim
import numpy as np
import argparse
import matplotlib
import matplotlib.pyplot as plt

import importlib
import utils
importlib.reload(utils)

from utils import make_flow, make_flow_zuko, plot_individual_marginals


trial_dir ='/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Uncertainty_Modeling/Train_Models/zuko/EstimationNFzuko_outputs/N_100000_seeds_100_4_16_512_15'
f_i_file = os.path.join(trial_dir, "f_i.pth")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #sets device to GPU if available 
f_i_statedicts = torch.load(f_i_file, map_location=device) #list of state_dicts 


with open(os.path.join(trial_dir, "architecture_config.json")) as f:
    config = json.load(f) 

backend = config.get("backend")  # fallback to nflows if not specified

data_path='/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Uncertainty_Modeling/Generate_Data/saved_generated_target_data/100k_target_training_set.npy'
x_data = torch.from_numpy(np.load(data_path)).float().to(device)

# ------------------
# Reconstruct flows f_i
# ------------------
f_i_models = []
for state_dict in f_i_statedicts:
    if backend == "zuko":
        flow = make_flow_zuko(
            num_layers=config["num_layers"],
            hidden_features=config["hidden_features"],
            num_bins=config["num_bins"],
            num_blocks=config["num_blocks"],
            num_features=2,
            num_context=0,
            bayesian=config["bayesian"]
        ).to(device)
    else:
        flow = make_flow(
            num_layers=config["num_layers"],
            hidden_features=config["hidden_features"],
            num_bins=config["num_bins"],
            num_blocks=config["num_blocks"]
        ).to(device)

    flow.load_state_dict(state_dict)
    flow.eval()
    f_i_models.append(flow)



out_path='/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Uncertainty_Modeling/Train_Models/zuko/EstimationNFzuko_outputs/N_100000_seeds_100_4_16_512_15'
out_dir = os.path.join(out_path, "marginals")
os.makedirs(out_dir, exist_ok=True)

feature_names = ["Feature 1", "Feature 2"]
num_samples=1000
plot_individual_marginals(
    f_i_models[89:],
    x_data,
    feature_names,
    out_dir,
    num_samples,
    use_zuko=(backend == "zuko")
)

FileNotFoundError: [Errno 2] No such file or directory: '/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Uncertainty_Modeling/Train_Models/zuko/EstimationNFzuko_outputs/N_100000_seeds_100_4_16_512_15/f_i.pth'